In [90]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.ensemble import RandomForestClassifier


In [91]:
df = pd.read_csv("../Data/Tiki_product_data_20230620.csv")
df.head()

,id,name,existed_day_count,price_kvnd,list_price_kvnd,original_price_kvnd,discount_kvnd,discount_rate,rating_average,review_count,quantity_sold_count,seller_count,category
0,53434797,Sách Tài Chính Cá Nhân Cho Người Việt Nam - Tặ...,1147,184.0,200.0,200.0,16.0,8,4.7,1754,13092,2,"Sách tài chính, tiền tệ"
1,75953557,Tâm Lý Học Về Tiền,888,110.0,189.0,189.0,79.0,42,4.8,3365,22842,17,"Sách tài chính, tiền tệ"
2,126853168,Thấu hiểu Hành vi giá Thị trường Tài chính (Un...,627,367.0,367.0,367.0,0.0,0,5.0,248,1459,2,"Sách tài chính, tiền tệ"
3,146223395,Cách thức kinh doanh và đầu cơ cổ phiếu! – How...,561,349.0,349.0,349.0,0.0,0,4.8,762,3824,1,"Sách tài chính, tiền tệ"
4,53532186,Tuyệt kỹ Giao dịch bằng đồ thị nến Nhật – Japa...,1146,499.0,499.0,499.0,0.0,0,4.8,2263,12664,1,"Sách tài chính, tiền tệ"


In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14399 entries, 0 to 14398
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   14399 non-null  int64  
 1   name                 14399 non-null  object 
 2   existed_day_count    14399 non-null  int64  
 3   price_kvnd           14399 non-null  float64
 4   list_price_kvnd      14399 non-null  float64
 5   original_price_kvnd  14399 non-null  float64
 6   discount_kvnd        14399 non-null  float64
 7   discount_rate        14399 non-null  int64  
 8   rating_average       14399 non-null  float64
 9   review_count         14399 non-null  int64  
 10  quantity_sold_count  14399 non-null  int64  
 11  seller_count         14399 non-null  int64  
 12  category             14399 non-null  object 
dtypes: float64(5), int64(6), object(2)
memory usage: 1.4+ MB


In [93]:
# Function to clean the word of any punctuation or special characters
def cleanpunc(sentence): 
    cleaned = re.sub(r'\W',r' ',sentence)
    return  cleaned

df['name_clean'] = df['name'].str.lower()
df['name_clean'] = df['name_clean'].apply(cleanpunc)
#df['name_clean'] = df['name_clean'].str.replace('tái bản', '')
#df['name_clean'] = df['name_clean'].str.replace('bản quyền', '')
df['name_clean']

0        sách tài chính cá nhân cho người việt nam   tặ...
1                                       tâm lý học về tiền
2        thấu hiểu hành vi giá thị trường tài chính  un...
3        cách thức kinh doanh và đầu cơ cổ phiếu    how...
4        tuyệt kỹ giao dịch bằng đồ thị nến nhật   japa...
                               ...                        
14394         top những cuốn sách hot nhất về bất động sản
14395                      cuốn sách  tinh anh công sở 4 0
14396    combo 2 cuốn sách gối đầu để thành công trong ...
14397                  thông minh hơn  nhanh hơn  giỏi hơn
14398                        trong chớp mắt  tái bản 2015 
Name: name_clean, Length: 14399, dtype: object

In [94]:
LE = LabelEncoder()
df['category_id'] = LE.fit_transform(df['category'])

In [95]:
# Number of words in book name
NUMBER_OF_COMPONENTS = 350

vectorizer = TfidfVectorizer(lowercase = True, analyzer='word', ngram_range = (1, 2), 
                             min_df = 0.01, max_df = 0.9)
svd_truncer = TruncatedSVD(n_components = NUMBER_OF_COMPONENTS)

name_mat = vectorizer.fit_transform(df['name_clean'])
name_mat = svd_truncer.fit_transform(name_mat)

In [99]:
train_x, test_x, train_y, test_y = train_test_split(name_mat, df['category_id'], random_state = 22)

rfc = RandomForestClassifier(random_state = 22, max_depth = 10, n_estimators = 100)

model_rfc = rfc.fit(train_x, train_y) 

train_yhat = rfc.predict(train_x)


In [100]:
print('Report of the model')
print(classification_report(train_y.values, train_yhat))

Report of the model
              precision    recall  f1-score   support

           0       0.54      0.81      0.65      1505
           1       0.80      0.81      0.81      1498
           2       0.96      0.21      0.34       777
           3       0.90      0.45      0.60       871
           4       0.88      0.59      0.71      1234
           5       0.50      0.82      0.62      1516
           6       0.94      0.40      0.56       329
           7       0.78      0.79      0.79      1465
           8       0.99      0.58      0.73       248
           9       0.82      0.80      0.81      1356

    accuracy                           0.69     10799
   macro avg       0.81      0.63      0.66     10799
weighted avg       0.76      0.69      0.69     10799



In [101]:
test_yhat = rfc.predict(test_x)
print('Report of the model')
print(classification_report(test_y.values, test_yhat))

Report of the model
              precision    recall  f1-score   support

           0       0.35      0.54      0.43       484
           1       0.67      0.74      0.71       498
           2       0.72      0.08      0.15       284
           3       0.82      0.41      0.55       294
           4       0.69      0.38      0.49       407
           5       0.34      0.61      0.44       475
           6       0.84      0.25      0.39        83
           7       0.64      0.63      0.64       521
           8       0.90      0.49      0.64        75
           9       0.65      0.66      0.65       479

    accuracy                           0.53      3600
   macro avg       0.66      0.48      0.51      3600
weighted avg       0.61      0.53      0.53      3600

